# **식품 가격변동 수집**
농산물 실시간 가격정보 API 제공 **Service** 목록
1. **[aT KAMIS](https://www.kamis.or.kr/customer/reference/openapi_list.do?action=detail&boardno=1)** 중심
1. **[서울시 농산물 유통공사](https://www.garak.co.kr/main/main.do)** 가격 정보 (단 축산물 미제공)
1. **[DATA.go.kr](https://www.data.go.kr/dataset/15012298/openapi.do)** 와 **[농산물 코드](http://data.mafra.go.kr/opendata/data/indexOpenDataDetail.do?data_id=20141220000000000380&service_ty=G&filter_ty=G&sort_id=regist_dt&s_data_nm=%ED%92%88%EB%AA%A9&cl_code=&instt_id=201410120010&shareYn=)**
1. **[축산물 품질 평가원](http://www.ekapepia.com/priceStat/period/periodPeriodPrice.do?menuId=menu100009&boardInfoNo=)**

### **추가로 날씨 API 정보수집 및 시각화**
http://www.weather.go.kr/weather/lifenindustry/sevice_rss.jsp

## **1 aT KAMIS 가격정보**
**일별 부류별 도.소매가격정보**
1. ID : **246**
1. apiKey : **fb54dcd7-218f-4297-8fa6-31cfcd0a897d**
```python
url = """
    http://www.kamis.or.kr/service/price/xml.do?action=dailyPriceByCategoryList
    &p_product_cls_code=01    &p_country_code=1101
    &p_regday=2019-12-11      &p_convert_kg_yn=Y
    &p_item_category_code=500
    &p_cert_key=111           &p_cert_id=222
    &p_returntype=json"""
```

In [5]:
url_head = "http://www.kamis.or.kr/service/price/xml.do?action=dailyPriceByCategoryList&"
query = {
    "p_product_cls_code":"01",  # 구분코드(01:소매, 02[기본]:도매)
    "p_country_code":"1101",    # 도소매(1101:서울, 2100:부산, 2200:대구, 2401:광주, 2501:대전) 
    "p_regday":"2019-12-11",    # 조사일자
    "p_convert_kg_yn":"N",      # kg 환산(Y:1kg, N[기본]:조사단위)
    "p_item_category_code":"200", # (100[기본]:식량,200:채소,300:특용,400:과일,500:축산,600:수산)
    "p_cert_key":"111",
    "p_cert_id":"222",
    "p_returntype":"json" }

import json
import pandas as pd
from urllib import parse, request
url = url_head + parse.urlencode(query)
resp = json.loads(request.urlopen(url).read().decode('utf-8'))['data']
if type(resp) != dict:
    print("Parsing Error :", resp)
else: 
    resp_table = pd.DataFrame(resp['item'])
    # parsed_table.to_csv("data/atkamis.csv", index=None)
    print(resp_table.head(2))

  item_name item_code kind_name kind_code rank rank_code unit        day1  \
0        배추       211   가을(1포기)        03   상품        04  1포기  당일 (12/11)   
1        배추       211   가을(1포기)        03   중품        05  1포기  당일 (12/11)   

    dpr1         day2  ...          day3   dpr3          day4   dpr4  day5  \
0  4,433  1일전 (12/10)  ...  1주일전 (12/04)  4,475  2주일전 (11/27)  4,740  1개월전   
1  3,694  1일전 (12/10)  ...  1주일전 (12/04)  3,493  2주일전 (11/27)  3,417  1개월전   

    dpr5 day6   dpr6 day7   dpr7  
0  4,827  1년전  3,060  일평년  2,346  
1  3,834  1년전  2,373  일평년  2,114  

[2 rows x 21 columns]


# Class funciton

In [8]:
import json, time
import pandas as pd
from datetime import date
from urllib import parse, request
class ATKamis:
    def __init__(self):
        self.head = "http://www.kamis.or.kr/service/price/xml.do?action=dailyPriceByCategoryList&"
        self.query = {
            "p_country_code":"1101", # 지역 (1101:서울, 2100:부산, 2200:대구, 2401:광주, 2501:대전) 
            "p_convert_kg_yn":"N",   # kg 환산(Y:1kg, N[기본]:조사단위)
            "p_cert_key":"111",
            "p_cert_id":"222",
            "p_returntype":"json" }

    # 일자별 모든 데이터 수집하기
    def getJson(self, cls="01", day=date.today().isoformat(), category="100"):
        r"""at Kamis 일자별 가격수집기
        :param cls: 도/소매 구분 (01/02)
        :param day: 수집기준 날짜 '2020-01-01'
        :param category: 100(식량)200(채소)300(특용)400(과일)500(축산)600(수산)"""
        
        query = self.query
        query["p_product_cls_code"] = cls
        query["p_regday"] = day                   # "2019-12-11"
        query["p_item_category_code"] = category  # "200"
        url = self.head + parse.urlencode(query)
        result_data = json.loads(request.urlopen(url).\
                                 read().decode('utf-8'))['data']
        if result_data != ['001']:
            return result_data['item']
        else: print("ERROR!! {} 도소매:{} 코드:{} {}".format(
            day, cls, category, result_data))
    
    # 일자별 전체 데이터 수집
    def getDayData(self, day=date.today()):
        result = []
        for _ in ["100","200","300","400","500","600"]:
            time.sleep(0.03)
            result.append(pd.DataFrame(
                self.getJson(cls="01", day=day, category=_)))
            result.append(pd.DataFrame(
                self.getJson(cls="02", day=day, category=_)))
        return pd.concat(result).reset_index(drop=True)

In [7]:
# ATKamis().getJson(cls='08', day='2019-12-04')
df_dayTable = ATKamis().getDayData()
df_dayTable.head(3)

ERROR!! 2019-12-16 도소매:02 코드:500 ['001']


,item_name,item_code,kind_name,kind_code,rank,rank_code,unit,day1,dpr1,day2,...,day3,dpr3,day4,dpr4,day5,dpr5,day6,dpr6,day7,dpr7
0,쌀,111,일반계(20kg),01,상품,04,20kg,당일 (12/16),-,1일전 (12/15),...,1주일전 (12/09),"51,663",2주일전 (12/02),"51,525",1개월전,"51,305",1년전,"52,999",일평년,"43,677"
1,찹쌀,112,일반계(1kg),01,상품,04,1kg,당일 (12/16),-,1일전 (12/15),...,1주일전 (12/09),"4,415",2주일전 (12/02),"4,390",1개월전,"4,551",1년전,"4,578",일평년,"4,180"
2,콩,141,흰 콩(국산)(500g),01,상품,04,500g,당일 (12/16),-,1일전 (12/15),...,1주일전 (12/09),"4,994",2주일전 (12/02),"4,994",1개월전,"4,994",1년전,"4,823",일평년,"4,538"
